# Setup

Import packages

In [1]:
import pandas
import pickle
from tqdm import tqdm

from sklearn.model_selection import train_test_split

Support constants

In [2]:
#constants
RANDOM_SEED = 42
TEST_SPLIT = 0.2

Load original data

In [3]:
clean_data = pandas.read_csv('data/clean_data.csv')
clean_data.head()

,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,Climate_Region_Pub,...,SCALEKER,IECC_Climate_Pub,HDD50,CDD80,GND_HDD65,WSF,OA_LAT,GWT,DesignDBT99,DesignDBT1
0,2,4,12,2,2471.68,4742,1080,4953,1271,4,...,-2,4A,2117,56,4250,0.48,6,56,9,96
1,4,10,26,2,8599.17,2662,199,2688,143,5,...,-2,3C,62,26,2393,0.61,0,64,38,73
2,1,1,1,5,8969.92,6233,505,5741,829,1,...,-2,5A,2346,49,5654,0.48,3,52,12,88
3,2,3,7,2,18003.64,6034,672,5781,868,1,...,-2,5A,2746,0,4941,0.55,4,55,7,87
4,1,1,1,3,5999.61,5388,702,5313,797,1,...,-2,5A,2251,0,5426,0.61,4,50,13,90


In [4]:
layout = pandas.read_csv('data/public_layout.csv', index_col='Variable Name')
#layout.reset_index(level=0, inplace=True)
layout.head()

,Variable Label,Variable Order in File,Variable Type,Length
Variable Name,,,,
DOEID,Unique identifier for each respondent,1,Character,5
REGIONC,Census Region,2,Numeric,8
DIVISION,Census Division,3,Numeric,8
REPORTABLE_DOMAIN,Reportable states and groups of states,4,Numeric,8
TYPEHUQ,Type of housing unit,5,Numeric,8


In [5]:
# Prediction target is
target = 'KWH'
layout['Variable Label'][target]

'Total Site Electricity usage, in kilowatt-hours, 2009'

# Data preparation

In [6]:
target_series = clean_data[target]
work_data = clean_data.drop(target, axis=1)

In [7]:
one_hot_encoding = pandas.read_csv('data/recs2009_public.csv', usecols=['DOEID'])
for c in layout.index[layout['Variable Type']=='Character']: 
    if c in work_data.columns:
        one_hot_encoding = one_hot_encoding.join(pandas.get_dummies(work_data[c], prefix=c, dummy_na=True))
        work_data = work_data.drop(c, axis=1)
one_hot_encoding = one_hot_encoding.drop('DOEID', axis=1)
one_hot_encoding.head()

,METROMICRO_METRO,METROMICRO_MICRO,METROMICRO_NONE,METROMICRO_nan,UR_R,UR_U,UR_nan,IECC_Climate_Pub_1A-2A,IECC_Climate_Pub_2B,IECC_Climate_Pub_3A,IECC_Climate_Pub_3B-4B,IECC_Climate_Pub_3C,IECC_Climate_Pub_4A,IECC_Climate_Pub_4C,IECC_Climate_Pub_5A,IECC_Climate_Pub_5B-5C,IECC_Climate_Pub_6A-6B,IECC_Climate_Pub_7A-7B-7AK-8AK,IECC_Climate_Pub_nan
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
work_data.head()

,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,Climate_Region_Pub,...,PERIODKR,SCALEKER,HDD50,CDD80,GND_HDD65,WSF,OA_LAT,GWT,DesignDBT99,DesignDBT1
0,2,4,12,2,2471.68,4742,1080,4953,1271,4,...,-2,-2,2117,56,4250,0.48,6,56,9,96
1,4,10,26,2,8599.17,2662,199,2688,143,5,...,-2,-2,62,26,2393,0.61,0,64,38,73
2,1,1,1,5,8969.92,6233,505,5741,829,1,...,-2,-2,2346,49,5654,0.48,3,52,12,88
3,2,3,7,2,18003.64,6034,672,5781,868,1,...,-2,-2,2746,0,4941,0.55,4,55,7,87
4,1,1,1,3,5999.61,5388,702,5313,797,1,...,-2,-2,2251,0,5426,0.61,4,50,13,90


Normalize numeric features

In [9]:
tqdm.pandas(desc='Feature normalization')
work_data = work_data.progress_apply(lambda x: (x - x.mean())/x.std() )

C:\Users\vbfal\.conda\envs\ProjectZ\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Feature normalization: 100%|███████████████████████████████████████████████████████| 544/544 [00:00<00:00, 1103.47it/s]


In [10]:
work_data = work_data.join(one_hot_encoding).copy()
work_data = work_data.join(target_series).copy()
work_data

,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,Climate_Region_Pub,...,IECC_Climate_Pub_3B-4B,IECC_Climate_Pub_3C,IECC_Climate_Pub_4A,IECC_Climate_Pub_4C,IECC_Climate_Pub_5A,IECC_Climate_Pub_5B-5C,IECC_Climate_Pub_6A-6B,IECC_Climate_Pub_7A-7B-7AK-8AK,IECC_Climate_Pub_nan,KWH
0,-0.603243,-0.480465,-0.339943,-0.552874,-1.334851,0.259305,-0.295721,0.361971,-0.168957,1.036457,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,18466
1,1.316109,1.618052,1.368449,-0.552874,-0.154789,-0.638115,-1.072141,-0.640015,-1.272915,1.777458,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5148
2,-1.562919,-1.529724,-1.682251,1.963175,-0.083388,0.902599,-0.802465,0.710565,-0.601536,-1.186546,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2218
3,-0.603243,-0.830218,-0.950083,-0.552874,1.656371,0.816740,-0.655289,0.728260,-0.563368,-1.186546,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10015
4,-1.562919,-1.529724,-1.682251,0.285809,-0.655425,0.538022,-0.628850,0.521227,-0.632854,-1.186546,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12076,-0.603243,-0.830218,-0.706027,-0.552874,0.157320,1.523026,-0.954929,1.614343,-0.971480,-1.186546,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7647
12077,0.356433,0.568794,0.758309,1.124492,-0.137475,-0.887062,1.258881,-0.714334,0.824410,-0.445545,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1813
12078,-0.603243,-0.480465,-0.583999,-0.552874,-0.906248,1.636929,-0.831548,1.742632,-0.825656,-1.186546,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9834
12079,1.316109,0.918547,1.002365,-0.552874,0.722727,0.730450,-0.568922,0.820274,-0.476265,-1.186546,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4800


In [11]:
# Validate no problems were created
work_data.isna().any().any()

False

We're good to proceed!

# Dataset split

In [12]:
(train_index, test_index) = train_test_split(work_data.index, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

train_data = work_data.iloc[train_index]
test_data = work_data.iloc[test_index]

train_data.reset_index(inplace = True, drop = True)
test_data.reset_index(inplace = True, drop = True)

In [13]:
train_data.to_csv('data/train_data.csv', index=False)
test_data.to_csv('data/test_data.csv', index=False)